In [ ]:
import cv2
import pickle
import numpy as np
import os

In [ ]:

facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
faces_data=[]

i=0

In [ ]:
name=input("Enter Your Name: ")

In [ ]:
video=cv2.VideoCapture(0)
while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img=cv2.resize(crop_img, (50,50))
        if len(faces_data)<=100 and i%10==0:
            faces_data.append(resized_img)
        i=i+1
        cv2.putText(frame, str(len(faces_data)), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (50,50,255), 1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
    cv2.imshow("Frame",frame)
    k=cv2.waitKey(1)
    if k==ord('q') or len(faces_data)==100:
        break
video.release()
cv2.destroyAllWindows()

In [ ]:
video.release()
cv2.destroyAllWindows()

In [ ]:
faces_data=np.asarray(faces_data)
faces_data=faces_data.reshape(100, -1)


if 'names.pkl' not in os.listdir('data/'):
    names=[name]*100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)
else:
    with open('data/names.pkl', 'rb') as f:
        names=pickle.load(f)
    names=names+[name]*100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)

if 'faces_data.pkl' not in os.listdir('data/'):
    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces_data, f)
else:
    with open('data/faces_data.pkl', 'rb') as f:
        faces=pickle.load(f)
    faces=np.append(faces, faces_data, axis=0)
    with open('data/faces_data.pkl', 'wb') as f:
        pickle.dump(faces, f)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime


from win32com.client import Dispatch

: 

In [ ]:
def speak(str1):
    speak=Dispatch(("SAPI.SpVoice"))
    speak.Speak(str1)


In [ ]:
video=cv2.VideoCapture(0)
facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

In [ ]:
import cv2
import pickle

facedetect = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

with open('data/names.pkl', 'rb') as w:
    LABELS = pickle.load(w)

with open('data/faces_data.pkl', 'rb') as f:
    FACES = pickle.load(f)


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# Assuming you have imported the necessary libraries and loaded the FACES and LABELS arrays

# Check the number of samples in each array
num_samples_faces = len(FACES)
num_samples_labels = len(LABELS)

# Take the minimum number of samples
num_samples = min(num_samples_faces, num_samples_labels)

# Truncate the arrays to have the same number of samples
FACES = FACES[:num_samples]
LABELS = LABELS[:num_samples]

# Create a KNN classifier object
knn = KNeighborsClassifier(n_neighbors=5)  # Specify the number of neighbors to consider (e.g., 3)

# Fit the classifier to the truncated training data
knn.fit(FACES, LABELS)


In [ ]:
import cv2

# Assuming you have imported the necessary libraries and have a background image named 'background.jpg'

# Load the background image
imgBackground = cv2.imread('background.jpg')

# Check if the image is loaded successfully
if imgBackground is not None:
    pass
    # Perform further processing with the background image
    # For example, you can use it for image blending or background substitution

    # Rest of your code goes here, using imgBackground as needed

else:
    print("Failed to load the background image.")

# Make sure to handle any errors or exceptions that may occur during image loading or processing



In [ ]:
COL_NAMES = ['NAME', 'TIME']
video = cv2.VideoCapture(0)
while True:
    ret,frame=video.read(0)
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img=cv2.resize(crop_img, (50,50)).flatten().reshape(1,-1)
        output=knn.predict(resized_img)
        ts=time.time()
        date=datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp=datetime.fromtimestamp(ts).strftime("%H:%M-%S")
        exist=os.path.isfile("Attendance/Attendance_" + date + ".csv")
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
        cv2.putText(frame, str(output[0]), (x,y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
        attendance=[str(output[0]), str(timestamp)]
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame",imgBackground)
    k=cv2.waitKey(1)
    if k==ord('o'):
        speak("Attendance Taken..")
        time.sleep(5)
        if exist:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(attendance)
            csvfile.close()
        else:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
            csvfile.close()
    if k==ord('q'):
        break
video.release()
cv2.destroyAllWindows()